# REQUEST by PREDICT


Copy-paste the request created in Predict

In [ ]:
#Replace the two lines below with the result of your PREDICT request

req1 = " ( source_id == 'CESM2' ) & ( experiment_id == 'ssp585' ) & ( table_id == 'Amon' ) & ( variable_id == 'pr' ) "
req2 = "{ 'lons':[ -0.368031, 2.264177, 9.186397 ], 'lats':[ 39.445738, 48.827636, 45.458631 ]} "

In [ ]:
# Generate environment - it takes several minutes - If you run a second request, there is no need to run this piece of code again.
import pip
pip.main(["install","matplotlib", "pandas", "xarray", "zarr", "gcsfs", "cftime", "dask[array]", "toolz", "nc-time-axis", "openpyxl", " xlsxwriter"])

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import gcsfs
import cftime
import xlsxwriter
import dask
import toolz
import os
import ast


xr.set_options(display_style='html')
%matplotlib inline
%config InlineBackend.figure_format = 'retina' 

## Browse Catalog

The data catatalog is stored as a CSV file. Here we read it with Pandas.

In [ ]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df_ta = df.query(req1)


The columns of the dataframe correspond to the CMI6 controlled vocabulary. A beginners' guide to these terms is available in [this document](https://docs.google.com/document/d/1yUx6jr9EdedCOLd--CPdTfGDwEwzPpCF6p1jRmqx-0Q). 

## Load & Extract Data

Now we will load a single store using gcsfs, zarr, and xarray.

In [ ]:

df_out = {}
df_assets = {}
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('dataset_requested.xlsx', engine='xlsxwriter')


for i in range(df_ta.shape[0]):
    df_inter = []
    # this only needs to be created once
    gcs = gcsfs.GCSFileSystem(token='anon')

    # get the path to a specific zarr store (the first one from the dataframe above)
    zstore = df_ta.zstore.values[i]

    # create a mutable-mapping-style interface to the store
    mapper = gcs.get_mapper(zstore)

    # open it using xarray and zarr
    ds = xr.open_zarr(mapper, consolidated=True)
    print(ds)
    
    #Get Metadata
    ds_attrs = pd.DataFrame.from_dict(ds.attrs, orient ='index')
    
    #Filter on assets
    # intialise data of lists. 
    df_assets = pd.DataFrame(ast.literal_eval(req2))
  
    for j in range(len(df_assets)):
        print(j)
        ds_sample = ds.sel(lon = df_assets["lons"].iloc[[j]], lat = df_assets["lats"].iloc[[j]], method = 'nearest')
        #Convert to df
        df_1 = ds_sample.to_dataframe().reset_index()
        #get data to 2050
        df_1 = df_1[:13514]
        
        if j == 0 :
            df_inter = df_1
            print(df_inter)
        else :
            df_inter = df_inter.append(df_1)
            print(df_inter)
    
    #df_inter = df_inter[df_inter.columns[~df_inter.columns.str.contains('bnds')]]
    
    #Save in excel
    ds_attrs.to_excel(writer, sheet_name='Metadata_' + str(i) )
    df_inter.to_excel(writer, sheet_name='Raw_Data_' + str(i) )
    
writer.save()
